In [ ]:
%load_ext autoreload
%autoreload 2
from add_parent_to_sys_path import add_parent_to_sys_path
add_parent_to_sys_path()
from modules.sftp_ops import *
import json 
import logging
clear_logging_handlers()

#Configure loggging
logging.basicConfig(filename='../logs/SFTP_operations.log', level=logging.INFO,
                   format='%(asctime)s - %(message)s', datefmt='%d-%b-%y %H:%M:%S')
logging.info('\n\n-------------New SFTP Operations Logging Instance')


with open('../powerschool-420113-db919282054b.json') as json_file:
    j = json.load(json_file) 
    savva_password = j['savva_password']
    savva_username = j['savva_username']
    savva_host = j['savva_host']
    clever_export_password = j['clever_export_password']
    clever_export_username = j['clever_export_username']
    clever_export_host = j['clever_export_host']



#All BQ tables that are begin queryed are the keys. 
#How they are saved in the local dir are the values
clever_dictionary = {'Clever_schools':'schools.csv',
                    'Clever_students':'students.csv',
                    'Clever_teachers':'teachers.csv',
                    'Clever_sections':'sections.csv',
                    'Clever_enrollments':'enrollments.csv', 
                    'Clever_staff': 'staff.csv'
                    }

savva_dictionary = {
                    'SAVVAS_CODE_DISTRICT': 'CODE_DISTRICT.txt',
                    'SAVVAS_SCHOOL': 'SCHOOL.txt',
                    'SAVVAS_STUDENT': 'STUDENT.txt',
                    'SAVVAS_STAFF': 'STAFF.txt',
                    'SAVVAS_PIF_SECTION': 'PIF_SECTION.txt',
                    'SAVVAS_PIF_SECTION_STAFF': 'PIF_SECTION_STAFF.txt',
                    'SAVVAS_PIF_SECTION_STUDENT': 'PIF_SECTION_STUDENT.txt',
                    'SAVVAS_ENROLLMENT': 'ENROLLMENT.txt',
                    'SAVVAS_ASSIGNMENT': 'ASSIGNMENT.txt'
                }



sftp_conn_clever_export = SFTPConnection(
    host=clever_export_host,
    username=clever_export_username,
    password=clever_export_password,
    use_pool=False
)

#Export BQ views to local dir. Follow naming convention of dictionary arg. 
SFTP_conn_file_exchange(sftp_conn_clever_export,
                        import_or_export = 'export',
                        sftp_folder_name='clever_iota_file_transfer', 
                        db='roster_files',
                        naming_dict = clever_dictionary,
                        use_pool=False,
                        )

# Export the local replicated files to Clevers SFTP
SFTP_export_dir_to_SFTP(local_dir=os.getcwd() + '\\clever_iota_file_transfer',
               remote_dir='/home/boundless-calendar-0789',  #root dir on clevers sftp
               sftp = sftp_conn_clever_export)


# ----------------------------SAVVA piece-----------------------------------

sftp_conn_savva = SFTPConnection(
    host=savva_host,
    username=savva_username,
    password=savva_password,
    use_pool=False
)


#Export replicates BQ views to local dir. Follow naming convention as dictionary arg. 
SFTP_conn_file_exchange(sftp_conn_savva,
                        import_or_export = 'export',
                        sftp_folder_name='savva_iota_file_transfer', 
                        db='roster_files',
                        naming_dict = savva_dictionary,
                        use_pool=False
                        )

#Sends local files over the SAVVAS sftp SIS folder
SFTP_export_dir_to_SFTP(local_dir=os.getcwd() + '\\savva_iota_file_transfer',
               remote_dir='/SIS',  #root dir on clevers sftp
               sftp = sftp_conn_savva)



sftp_conn_clever_export.close_all_connections()
sftp_conn_savva.close_all_connections()
logging.info('Process has reached the end')




#Current run diagnosis. 
#Remove piplock until ran in virtual env
#Implement a catch if the file does not exist in BQ, example is clever_enrollments
#Specify which sftp conn is established in logs


In [ ]:
# Hi Sam,

#Current SFTP operations file running at 5AM. 


# I'm working on some improvements to the timing and organization of our data processes.
# Can you make the changes below?
# For the file we are pulling from reports-sftp.clever.com:  idm-sensitive-exports
# Clever updates this each morning at 6:45 AM, so pull from their SFTP at 6:50 AM and send to GCS/BQ. 
# To keep it separate from the exports coming in from PS, let's put this in a separate dataset 
# (it can be called misc_imports).

# Also move the TN_options_report_2014 file form EasyIEP to misc_imports. 
# It can also be pulled at 6:50 if it's easiest to do those two together.
# I have a query set up that creates a list of the credentials that need
#  to be updated in PS - it drops those into
#  two tables in the powerschool_staged db: GoogleForPSImport_ASD and GoogleForPSImport_TPCSC. 
# Could we send these back to the IOTA SFTP at 7:05 as two separate .txt files? 
# Then I'll set up an automated import into each PS so that we have updated credentials before school starts.
#  I'll probably also have EasyIEP files to import too at some point in the future.
# Let me know if you have any questions or want to talk through!



#Seperate Clever import out idm-sensitive-exports make this occur at 6:50 AM label it in a folder called misc imports 
#Get rid of the older one, bucket, and dataset
#This piece is going to need to be updated for Google Cloud send as well. Seperate this piece out. 
#Same thing needs to occur for TN_options_report_2014. Put it in a misc imports at 6:50 AM. 
#Tested with the bigqueryoperations

#Issue with google-powerschool-students-passwords table creation. 
#Issue with this email being a column name aaguilon210421@iotaschools_org
#See if Amy needs this file, if so need to implement some Dynamic Error catching with optional args


#THis will be another seperate script
#Pull these Big Query tables down GoogleForPSImport_ASD and GoogleForPSImport_TPCSC send these to IOTA SFTP at 7:05
#This will eventually be expanded out. 


#Fix import based on folder movement
#Delete idm_sensitive_exports files out of Powerschool Combined
#Logs paths got changed

#Test out that nestd file paths are working
